In [1]:
import tensorflow as tf
import ktrain

if tf.test.gpu_device_name(): 

    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))

Default GPU Device:/device:GPU:0


In [2]:

import ktrain
from ktrain import text

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold

num_folds = 5

run_precision = []
run_recall = []
run_f1score = []
run_accuracy = []

kf = KFold(n_splits=5, shuffle=False, random_state=125)

In [4]:
data = pd.read_csv('sstubsIn_edited.csv')

In [5]:
print(data.shape)

(10231, 5)


In [6]:
#print("pre-processing train data...")

from sklearn.utils import shuffle
from tqdm import tqdm



raw_docs_train = data['sourceBeforeFix'].tolist()
#raw_docs_test = test_data['message'].tolist() 
num_classes = 4


word_seq_train = raw_docs_train
word_seq_train = np.array(word_seq_train)

#data.bugType.value_counts()

In [7]:
class_names = [0,1,2,3]

In [8]:
encoding = {
    'CHANGE_IDENTIFIER': 0, 
    'CHANGE_MODIFIER': 0,
    'CHANGE_NUMERAL': 0, 
    'SWAP_BOOLEAN_LITERAL': 0,
     'DIFFERENT_METHOD_SAME_ARGS': 1, 
     'OVERLOAD_METHOD_MORE_ARGS': 1, 
    'OVERLOAD_METHOD_DELETED_ARGS': 1,
     'CHANGE_CALLER_IN_FUNCTION_CALL': 1, 
     'SWAP_ARGUMENTS': 1,
     'CHANGE_OPERATOR': 2, 
    'CHANGE_UNARY_OPERATOR': 2, 
    'CHANGE_OPERAND': 2, 
     'LESS_SPECIFIC_IF': 3,
    'MORE_SPECIFIC_IF': 3,
     'ADD_THROWS_EXCEPTION': 3, 
      'DELETE_THROWS_EXCEPTION':3,
       
}

In [9]:
y_train_initial= data['bugType'].values


y_train_initial = [encoding[x] for x in y_train_initial]
y_train_initial=np.array(y_train_initial)

In [10]:
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=150, class_names=class_names)


In [11]:
import tensorflow as tf
from sklearn import metrics
from sklearn.model_selection import train_test_split
count = 1

for train_index, test_index in kf.split(word_seq_train):
    x_trn, x_tst = word_seq_train[train_index], word_seq_train[test_index]
    y_trn, y_tst = y_train_initial[train_index], y_train_initial[test_index]
    
    x_new_train, x_val, y_new_train, y_val= train_test_split(x_trn, y_trn, test_size=0.11115, random_state=125)
    
    print("\nFold ", count)
    
    
    
    trn = t.preprocess_train(x_new_train, y_new_train)
    val = t.preprocess_test(x_val, y_val)
    tst= t.preprocess_test(x_tst, y_tst)
    
    model = t.get_classifier()
    
    learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=12)
    
    callbacks = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=2)
    
    learner.lr_find()
     
    learner.fit_onecycle(2e-5, 40, callbacks=[callbacks])
   
    
    
    learner.validate(val_data=tst, class_names=class_names)
    
    '''
    print(metrics.classification_report(y_tst, y_pred))
    
    precision = precision_score(y_tst, y_pred, pos_label=1)
    recall = recall_score(y_tst, y_pred, pos_label=1)
    f1score = f1_score(y_tst, y_pred, pos_label=1)
    lstm_accuracy = accuracy_score(y_tst, y_pred)

    run_accuracy.append(accuracy)
    run_f1score.append(f1score)
    run_precision.append(precision)
    run_recall.append(recall)
    '''
    count = count+1
    #X_train = data['sourceBeforeFix'].tolist()


Fold  1
preprocessing train...
language: en
train sequence lengths:
	mean : 4
	95percentile : 9
	99percentile : 33


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 5
	95percentile : 8
	99percentile : 35


preprocessing test...
language: en
test sequence lengths:
	mean : 3
	95percentile : 8
	99percentile : 22


simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
606/606 [==============================] - 52s 78ms/step - loss: 1.2565 - accuracy: 0.4574
Epoch 2/1024
606/606 [==============================] - 49s 80ms/step - loss: 0.7718 - accuracy: 0.6564
Epoch 3/1024
606/606 [==============================] - 49s 81ms/step - loss: 1.0027 - accuracy: 0.6185
Epoch 4/1024
606/606 [==============================] - 3s 5ms/step - loss: 1.8513 - accuracy: 0.4672


done.
Please invoke the Learner.lr_plot() method to visually inspect the loss plot to help identify the maximal learning rate associated with falling loss.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/40
607/607 [==============================] - 58s 88ms/step - loss: 1.0765 - accuracy: 0.5861 - val_loss: 0.7649 - val_accuracy: 0.6165
Epoch 2/40
607/607 [==============================] - 54s 87ms/step - loss: 0.6736 - accuracy: 0.6664 - val_loss: 0.6828 - val_accuracy: 0

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 3
	95percentile : 9
	99percentile : 30


...

preprocessing test...
language: en
test sequence lengths:
	mean : 5
	95percentile : 9
	99percentile : 34


simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
606/606 [==============================] - 52s 77ms/step - loss: 1.2281 - accuracy: 0.5914
Epoch 2/1024
606/606 [==============================] - 47s 78ms/step - loss: 0.8085 - accuracy: 0.6387
Epoch 3/1024
606/606 [==============================] - 47s 78ms/step - loss: 0.9722 - accuracy: 0.6334
Epoch 4/1024
606/606 [==============================] - 3s 5ms/step - loss: 1.7350 - accuracy: 0.5173


done.
Please invoke the Learner.lr_plot() method to visually inspect the loss plot to help identify the maximal learning rate associated with falling loss.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/40
607/607 [==============================] - 55s 83ms/step - loss: 1.0574 - accuracy: 0.6137 - val_loss: 0.6825 - val_accuracy: 0.6692
Epoch 2/40
607/607 [==============================] - 50s 82ms/step - loss: 0.6658 - accuracy: 0.6888 - val_loss: 0.6198 - val_accuracy: 0

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 3
	95percentile : 10
	99percentile : 37


preprocessing test...
language: en
test sequence lengths:
	mean : 3
	95percentile : 8
	99percentile : 22


simulating training for different learning rates... this may take a few moments...
Epoch 1/1024
606/606 [==============================] - 51s 77ms/step - loss: 1.2904 - accuracy: 0.3619
Epoch 2/1024
606/606 [==============================] - 47s 78ms/step - loss: 0.7503 - accuracy: 0.6464
Epoch 3/1024
606/606 [==============================] - 47s 78ms/step - loss: 1.0636 - accuracy: 0.6122
Epoch 4/1024
606/606 [==============================] - 3s 5ms/step - loss: 1.8214 - accuracy: 0.4976


done.
Please invoke the Learner.lr_plot() method to visually inspect the loss plot to help identify the maximal learning rate associated with falling loss.


begin training using onecycle policy with max lr of 2e-05...
Epoch 1/40
607/607 [==============================] - 55s 83ms/step - loss: 1.0920 - accuracy: 0.5306 - val_loss: 0.6776 - val_accuracy: 0.6582
Epoch 2/40
607/607 [==============================] - 50s 82ms/step - loss: 0.6659 - accuracy: 0.6748 - val_loss: 0.5778 - val_accuracy: 0